In [1]:
import numpy as np
import os

In [6]:
def load_data(folder_path):
    x_train = np.load(os.path.join(folder_path, 'x_train.npy'))
    y_train = np.load(os.path.join(folder_path, 'y_train.npy'))
    x_test = np.load(os.path.join(folder_path, 'x_test.npy'))
    y_test = np.load(os.path.join(folder_path, 'y_test.npy'))
    return x_train, y_train, x_test, y_test

In [ ]:
x_train, y_train, x_test, y_test = load_data('lr4_dataset/')

В данной лабораторной работе будет практиковаться поиск гиперпараметров. Буду рассмотрены алгоритмы поиска гиперпараметров: grid search, random search.

Помимо поиска гиперпараметров будет рассмотрен алгоритм кросс-валидации, позволяющий получить более достоверную оценку качества модели в условиях недостатка данных.
Хотя в работе предоставлена тестовая выборка, она имеет сугубо теоретический характер (для получения финальной оценки) и на практике как правило недоступна. Поэтому во время подбора гиперпараметров используются лишь `x_train, y_train`. `x_test, y_test` используются лишь для получения финальной оценки, чтобы можно было видеть разницу между разными алгоритмами подбора гиперпараметров (если она будет).

Выберите одну модель из списка: MLPClassifier, SGDClassifier, DecisionTreeClassifier, RandomForestClassifier, SVC.
Для выбранной модели произведите поиск оптимальных гиперпараметров. Требование: поиск должен идти как минимум для двух гиперпараметров.

## 0. Обучение бейзлайн модели для проведения сравнения

In [4]:
print(f"Тренировочная выборка имеет {x_train.shape[0]} экземпляров и {x_train.shape[1]} признаков")
for count,num_class in zip(np.bincount(y_train),range(len(np.bincount(y_train)))):
    print(f"Класса: {num_class} число элементов: {count}")

Тренировочная выборка имеет 110 экземпляров и 256 признаков
Класса: 0 число элементов: 11
Класса: 1 число элементов: 11
Класса: 2 число элементов: 11
Класса: 3 число элементов: 11
Класса: 4 число элементов: 11
Класса: 5 число элементов: 11
Класса: 6 число элементов: 11
Класса: 7 число элементов: 11
Класса: 8 число элементов: 11
Класса: 9 число элементов: 11


In [5]:
print(f"Тестовая выборка имеет {x_test.shape[0]} экземпляров и {x_test.shape[1]} признаков")
for count,num_class in zip(np.bincount(y_test),range(len(np.bincount(y_test)))):
    print(f"Класса: {num_class} число элементов: {count}")

Тестовая выборка имеет 30 экземпляров и 256 признаков
Класса: 0 число элементов: 3
Класса: 1 число элементов: 3
Класса: 2 число элементов: 3
Класса: 3 число элементов: 3
Класса: 4 число элементов: 3
Класса: 5 число элементов: 3
Класса: 6 число элементов: 3
Класса: 7 число элементов: 3
Класса: 8 число элементов: 3
Класса: 9 число элементов: 3


In [2]:
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold

In [7]:
# Обучите бейзлайн модель без изменения гиперпараметров (т.е. используются гиперпараметры по умолчанию).
# Используйте результаты для дальнешейго анализа результатов.
baseline_model = DecisionTreeClassifier()
baseline_model.fit(x_train,y_train)
y_pred_baseline = baseline_model.predict(x_test)
# Напишите ваш код здесь.

In [8]:
accuracy_score_baseline_model = accuracy_score(y_test, y_pred_baseline)

In [9]:
print(
        f"Результаты baseline_model на тестовой выборке:\nЧисло экземпляров всех трех классов {np.bincount(y_pred_baseline)}")
print(
        f"\nЧисло реальных экземпляров тестовой выборке:\nЧисло экземпляров всех трех классов {np.bincount(y_test)}")
print(f"Точность baseline_model -> {accuracy_score_baseline_model}")

Результаты baseline_model на тестовой выборке:
Число экземпляров всех трех классов [5 3 0 1 4 8 0 6 0 3]

Число реальных экземпляров тестовой выборке:
Число экземпляров всех трех классов [3 3 3 3 3 3 3 3 3 3]
Точность baseline_model -> 0.4


## 1. K-Fold Cross-Validation

In [10]:
def model_factory(param_dict=None):
    if param_dict == None: return DecisionTreeClassifier()
    return DecisionTreeClassifier(**param_dict)

def accuracy_eval(labels, predictions):
    return accuracy_score(labels, predictions)

In [11]:
# Реализуйте фунцию кросс-валидации

# Замечание: x_test, y_test не должны применятся в рамках данной функции.

def kfold_cv(model_fn, eval_fn, x: np.ndarray, y: np.ndarray, n_splits=5, param_dict=None) -> float:
    """
    Parameters
    ----------
    model_fn : callable
        Функция-фабрика, что конструирует и возвращает новый объект модели.
    eval_fn : callable
        Функция вида `eval_fn(labels, predictions)`, что возвращает скаляр (значение метрики).
    x : np.ndarray
        Набор признаков (размерность NxD, N - количество экземпляро, D - количество признаков).
    y : np.ndarray
        Набор меток (размерность N)
    n_splits : int, optional
        Количество фолдов, по умолчанию 5.

    Returns
    -------
    float
        Среднее значение метрики (что вычисляется eval_fn) по фолдам.
    """
    kf = KFold(n_splits=n_splits, shuffle=True)
    scores = []

    for train_index, val_index in kf.split(x):
        x_train_kfold, x_val = x[train_index], x[val_index]
        y_train_kfold, y_val = y[train_index], y[val_index]
        
        model = model_fn(param_dict)  
        model.fit(x_train_kfold, y_train_kfold)  

        y_pred = model.predict(x_val)  

        score = eval_fn(y_val, y_pred)  
        scores.append(score)
    
    mean_score = np.mean(scores)
    return mean_score

In [12]:
# Протестируйте функцию кросс-валидации. 
# Не производите поиск гиперпараметров, лишь убедитесь в корректности реализации фукнции kfold_cv.
x_train, y_train, x_test, y_test = load_data('lr4_dataset/')
mean_score_CV = kfold_cv(model_factory,accuracy_eval,x_train,y_train)
# Напишите ваш код здесь.

In [13]:
print(f"Точность  K-Fold Cross-Validation -> {mean_score_CV}")

Точность  K-Fold Cross-Validation -> 0.2636363636363636


## 2. Grid search

In [14]:
from itertools import product
import time

In [15]:
# 1. Реализуйте алгоритм поиска гиперпараметров grid search.
# 2. Запустите поиск гиперпараметров, замерьте время работы алгоритма.
# 3. Выведите найденные значения гиперпараметров и время работы.
# Замечание: x_test, y_test не должны применятся в рамках данного алгоритма.
# Замечание: убедитесь, что гиперпараметры по умолчанию включены в пространство поиска.
# Требование: используйте kfold_cv для получения значения метрики в рамках одной итерации.
x_train, y_train, x_test, y_test = load_data('lr4_dataset/')
param_grid = {
    'max_depth':         [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf':  [1, 2, 4]
}

all_params = list(product(*param_grid.values()))
all_params.append(None)

print(f"Кол-во возможных вариантов по сетке:{len(all_params)}")
best_params = None
best_score = 0.0
start_time = time.time()
for params in all_params:
    if params is not None:
        param_dict = dict(zip(param_grid.keys(), params))
    else:
        param_dict = params
    mean_score = kfold_cv(model_factory,accuracy_eval,x_train,y_train,5,param_dict)
    
    if mean_score > best_score:
        best_score = mean_score
        best_params = param_dict
end_time = time.time()
execution_time = end_time - start_time        
print(best_params,best_score)
# Напишите ваш код здесь.

Кол-во возможных вариантов по сетке:37
{'max_depth': None, 'min_samples_split': 10, 'min_samples_leaf': 2} 0.4818181818181818


In [16]:
# Используйте найденные гиперпараметры для обучения модели. 
# Протестируйте модель на x_test, y_test.
# Сравните полученные результаты с теми, что получены в пункте 0.
GS_model = DecisionTreeClassifier(**best_params)
GS_model.fit(x_train,y_train)
y_pred_GS_model = GS_model.predict(x_test)
# Напишите ваш код здесь.

In [17]:
accuracy_score_GS_model = accuracy_score(y_test, y_pred_GS_model)

In [18]:
print(
        f"Результаты Grid search_model на тестовой выборке:\nЧисло экземпляров всех трех классов {np.bincount(y_pred_GS_model)}")
print(
        f"\nЧисло реальных экземпляров тестовой выборке:\nЧисло экземпляров всех трех классов {np.bincount(y_test)}")
print(f"Точность Grid search_model -> {accuracy_score_GS_model}\nВремя работы: {execution_time:.2f} секунд\nЛучшие параметры Grid search_model: {best_params}")

Результаты Grid search_model на тестовой выборке:
Число экземпляров всех трех классов [2 3 1 2 3 6 2 5 1 5]

Число реальных экземпляров тестовой выборке:
Число экземпляров всех трех классов [3 3 3 3 3 3 3 3 3 3]
Точность Grid search_model -> 0.5333333333333333
Время работы: 4.49 секунд
Лучшие параметры Grid search_model: {'max_depth': None, 'min_samples_split': 10, 'min_samples_leaf': 2}


## 3. Random search

In [19]:
import random

In [20]:
# 1. Реализуйте алгоритм поиска гиперпараметров random search.
# 2. Запустите поиск гиперпараметров, замерьте время работы алгоритма.
# 3. Выведите найденные значения гиперпараметров и время работы.
# Замечание: x_test, y_test не должны применятся в рамках данного алгоритма.
# Замечание: убедитесь, что гиперпараметры по умолчанию включены в пространство поиска.
# Требование: используйте kfold_cv для получения значения метрики в рамках одной итерации.
# Требование: количество итераций должно быть меньше в сравнении с grid search.
param_grid = {
    'max_depth': [5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}



RS_best_params = None
RS_best_score = 0.0
start_time_RS = time.time()


for _ in range(30):
    my_value = [random.randint(5, 20), random.randint(2, 10), random.randint(1, 5)]
    params = {param: value for param,value in zip(param_grid.keys(),my_value)}
    mean_score = kfold_cv(model_factory,accuracy_eval,x_train,y_train,5,params)
    if mean_score > RS_best_score:
        RS_best_score = mean_score
        RS_best_params = param_dict
        
mean_score = kfold_cv(model_factory,accuracy_eval,x_train,y_train,5,None)
if mean_score > RS_best_score:
    RS_best_score = mean_score
    RS_best_params = param_dict
end_time_RS = time.time()
execution_time_RS = end_time_RS - start_time_RS
print(RS_best_params,RS_best_score)

# Напишите ваш код здесь.

None 0.4909090909090909


In [21]:
# Используйте найденные гиперпараметры для обучения модели. 
# Протестируйте модель на x_test, y_test.
# Сравните полученные результаты с теми, что получены в пункте 0.
# Сравните полученные результаты с теми, что получены в пункте 2.`
if RS_best_params == None:
    RS_model = DecisionTreeClassifier()
else: 
    RS_model = DecisionTreeClassifier(**RS_best_params)
RS_best_params = RS_model.get_params()
RS_model.fit(x_train,y_train)
y_pred_RS = RS_model.predict(x_test)
# Напишите ваш код здесь.

In [22]:
RS_best_params = {'max_depth': RS_best_params['max_depth'],
                'min_samples_split': RS_best_params['min_samples_split'],
                    'min_samples_leaf': RS_best_params['min_samples_leaf']}

In [23]:
accuracy_score_RS = accuracy_score(y_test, y_pred_RS)

In [24]:
print(
        f"Результаты Grid search_model на тестовой выборке:\nЧисло экземпляров всех трех классов {np.bincount(y_pred_RS)}")
print(
        f"\nЧисло реальных экземпляров тестовой выборке:\nЧисло экземпляров всех трех классов {np.bincount(y_test)}")
print(f"Точность Random search_model -> {accuracy_score_RS}\nВремя работы: {execution_time_RS:.2f} секунд\nЛучшие параметры Random search_model:{RS_best_params}")

Результаты Grid search_model на тестовой выборке:
Число экземпляров всех трех классов [2 3 1 0 6 8 2 5 1 2]

Число реальных экземпляров тестовой выборке:
Число экземпляров всех трех классов [3 3 3 3 3 3 3 3 3 3]
Точность Random search_model -> 0.5
Время работы: 3.72 секунд
Лучшие параметры Random search_model:{'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 1}


In [25]:
print(f"Точность baseline_model -> {accuracy_score_baseline_model:.3f}\n")
print(f"Точность  K-Fold Cross-Validation -> {mean_score_CV:.3f}\n")
print(f"Точность Random search_model -> {accuracy_score_RS:.3f}\nВремя работы: {execution_time_RS:.2f} секунд\nЛучшие параметры Random search_model:{RS_best_params}\n")
print(f"Точность Grid search_model -> {accuracy_score_GS_model:.3f}\nВремя работы: {execution_time:.2f} секунд\nЛучшие параметры Grid search_model: {best_params}")

Точность baseline_model -> 0.400

Точность  K-Fold Cross-Validation -> 0.264

Точность Random search_model -> 0.500
Время работы: 3.72 секунд
Лучшие параметры Random search_model:{'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 1}

Точность Grid search_model -> 0.533
Время работы: 4.49 секунд
Лучшие параметры Grid search_model: {'max_depth': None, 'min_samples_split': 10, 'min_samples_leaf': 2}


Из результов видно, что Grid Search показал лучший результат, но требовал больше времени по сравнению с Random Search. 

## 4. Доп. задание (опционально)

### 4.1 Bayesian optimization

Примените байесовскую оптимизацию для поиска гиперпараметров.
В качестве алгоритма используйте `BayesSearchCV` из пакета `scikit-optimize`.

Сложность: почти бесплатный балл.

In [ ]:
# Используйте найденные гиперпараметры для обучения модели. 
# Протестируйте модель на x_test, y_test.
# Сравните полученные результаты с теми, что получены в пункте 0.
# Сравните полученные результаты с теми, что получены в пункте 2.

# Напишите ваш код здесь.

### 4.2 Tree of Parzen Estimators (TPE) из HyperOpt

Примените TPE из библиотеки hyperopt для поиска гиперпараметров. Вики по HyperOpt: https://github.com/hyperopt/hyperopt/wiki/FMin

Сложность: чтец документаций o(*￣▽￣*)ブ.

In [ ]:
def objective(args):
    # Принимает гиперпараметры, инстанцирует модель, обучает её, возвращает значение метрики.
    # Замечание: x_test, y_test не должны применятся в рамках данного алгоритма.
    
    # Напишите ваш код здесь.
    pass

In [ ]:
# Определите пространство поиска гиперпараметров
space = None

In [ ]:
# 1. Запустите поиск гиперпараметров, замерьте время работы алгоритма.
# 2. Выведите найденные значения гиперпараметров и время работы.

# Напишите ваш код здесь.

In [ ]:
# Используйте найденные гиперпараметры для обучения модели. 
# Протестируйте модель на x_test, y_test.
# Сравните полученные результаты с теми, что получены в пункте 0.
# Сравните полученные результаты с теми, что получены в пункте 2.

# Напишите ваш код здесь.